# Modeling by ray tracing

In this notebook, we show an example of using CSModeling to produce a synthetic data.

In [ ]:
# Import PyGêBR and gplot
from pygebr import *
from gplot import *
import requests

In [ ]:
# Build a flow to run CSModeling
csmodel = LoadFlow("/tmp/csmod.json", ['csmodeling'],
                   title="2D Acquisition",
                   description="Synthetic 2D survey with ray tracing modeling")

In [ ]:
# Open the UI to inspect and run the flow, with its default values
csmodel.ui()

In [ ]:
# See the model and the rays
gshow('/tmp/csmodel/out.svg')

# Many shots

Now, let us define some parameters to configure another flow to perform a 2D acquisition with many shots along the surface. The flow employing those parameters is already configured and will be download below.

In [ ]:
# Dictionary to hold parameters for following flows
p = {}

# Download the flow
URL = "https://gitlab.com/Biloti/pygebr/-/raw/main/samples/model02.json?inline=false"
flow = requests.get(URL)
with open("/tmp/model.json","w") as fp:
    fp.write(flow.text)

model = Flow(filename="/tmp/model.json", par = p)

In [ ]:
# Define many variables used to configure the flow
p['workdir'] = '/tmp'
p['modelname'] = '2dline'

path = p['workdir'] + "/" + p['modelname'] + "/"

p['headerfn'] = path + "out.hdr"
p['binfn'] = path + "out-0001.bin"
p['sufn'] = path + "out.su"
p['modelsvg'] = path + "out.svg"

p['tmax'] = 2.5
p['ns'] = int(p['tmax'] / 0.004 + 1)  # number of samples per trace (with 4ms)
p['firstshot'] = 100  # in meters
p['dstation'] = 50    # distance between stations, in meters
p['dshot'] = 200      # in meters
p['minoff'] = 100     # minimum offset
p['maxoff'] = 2000    # maximum offset

# Convert coordinates to stations (see csmodeling documentation)
firstst = int ((p['firstshot'] + p['minoff']) / p['dstation'])  # Most left station with receiver.
next1 = firstst+1                                               # This two stations are in sequence
next2 = firstst+2                                               # to indicate that there is no gap
lastst = int ((p['firstshot'] + p['maxoff']) / p['dstation'])   # Most right station with receiver.
shotst = p['firstshot'] / p['dstation']                         # Station where shot is located

# Line describing one shot geometry
p['shot'] = f"{firstst},{next1},{next2},{lastst},{shotst},0"

modelextension = 5000 # The model extends for 5km in this example
lastshotst = (modelextension - p['firstshot'])/p['dstation'] - (lastst-shotst)

# Total number of shots
p['nshots'] = int((lastshotst - shotst) * p['dstation'] / p['dshot'] + 1)

In [ ]:
# Define the output for the flow (filename and write mode)
model.stdout([p['sufn'], "w"])

# Load, inspect and run the flow.
# Note that many parameters were defined from the variables declared above.
model.ui()

In [ ]:
# Inspect the model and rays
gshow(p['modelsvg'])

In [ ]:
# To see a portion of the data
gimage(p['sufn'], title="2D Data set",
       label1="Time", unit1="s",
       label2="Trace",
       x2min=200, x2max=400,               # Inspect only a portion of the data
       palette=12
      )